In [ ]:
import configparser
import Src.data.dmbioProtDataset as dmbiods

In [ ]:
config_path = '/teamspace/studios/this_studio/DeepDive/Prot-DNA_experiments/test/testDatasetConfig.ini'
config = configparser.ConfigParser()
config.read(config_path)
config_section = config['DEFAULT']

In [ ]:
dsParams = dmbiods.dmbioProtDataSetParams(
    indir = config_section['indir'],
    in_file = config_section['in_file'],
    label_dir = config_section['label_dir'],
    label_fileExt = config_section['label_fileExt'],
    node_feat_dir = config_section['node_feat_dir'],
    node_feat_fileExt = config_section['node_feat_fileExt'],
    edge_dir = config_section['edge_dir'],
    edge_fileExt = config_section['edge_fileExt'],
    node_cord_dir = config_section['node_cord_dir'],
    node_cord_file_ext = config_section['node_cord_file_ext']
)

In [ ]:
datasset = dmbiods.dmbioProtDataSet(dsParams)

In [ ]:
datasset.getgraphbyid(0)